In [1]:
# imports
import os
import cv2
import sklearn.preprocessing as preproc
import numpy as np
import keras as k
#import matplotlib.pyplot as plt
#from time import time
import pandas as pd
import tensorflow as tf
from PIL import Image
from keras.preprocessing.image import ImageDataGenerator
import math
import shutil
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img

Using TensorFlow backend.


In [3]:
# dichiarazione funzioni

# codifica la colonna dx da originale (stringhe) a label encoding (0,1,..,6)
def lab_encode(dataset, le):
    new_ds = dataset
    new_ds["dx"] = le.fit_transform(dataset["dx"])  
    return new_ds


# decodifica colonna dx
def lab_decode(dataset, le):
    new_ds = dataset
    new_ds["dx"] = le.inverse_transform(dataset["dx"])  
    return new_ds


# conteggio immagini da generare dal dataset, risultato salvato in aug_size
def stats(dataset, aug_size):
    labels = set(dataset['dx'])
    for label in labels:
        print("class ", str(label), ':\t', str(dataset[dataset['dx']==label]['dx'].count()))
        x = dataset[dataset['dx']==label]['dx'].count()
        y = dataset[dataset['dx']==5]['dx'].count()
        value = math.floor((y-x)/x)
        print("ugmented per image: ", str(value), "\n")
        aug_size[label] = value
        
        
# apre il dataset originale, toglie le features superflue e lo ritorna insieme all'array delle labels
def initialize_dataset():
    dataset = pd.read_csv(csv_path, encoding = "ISO-8859-1")
    labels = set(dataset["dx"])
    new_ds = pd.DataFrame(dataset)
    columns=["lesion_id", "dx_type", "age", "sex", "localization"]
    new_ds = new_ds.drop(columns=columns, axis=0)
    return new_ds, labels


# dall'array di nomi (es. dataset["image_id"]) ritorna l'array d'immagini
def load_images(array):
    images = list()
    #print(array.iloc[0])
    for index in range(0,len(array)):
        #print(index.type())
        img_id = array.iloc[index]
        elem = os.path.join(image_path, img_id)
        elem = elem + ".jpg"
        img = cv2.imread(elem)
        images.append(np.asarray(img))
    return np.asarray(images)


#  ritorna la parte di dataset che nel nome contiene il nome delle tuple di originale (originale è un dataset senza immagini aumentate)
# (usato solo dopo aver partizionato originale)
def estendi(dataset, originale):
    res = pd.DataFrame()
    for index2 in range(len(originale)):
        temp = dataset[dataset["image_id"].str.contains(originale.iloc[index2]["image_id"])]
        res = res.append(temp)
        if index2 % 1000 == 0:
            print(index2)
    return res


# ritorna le 2 partizioni del dataframe (che non deve essere esteso) di fattori factor e 1-factor
# es. factor = 0.8
def partiziona(dataframe, factor):
    part1 = pd.DataFrame()  # factor originali
    part2 = pd.DataFrame()  # (1-factor) originali
    for elem in range(7):
        n = dataframe[dataframe['dx']==elem]['dx'].count() 
        p8 = int(n*factor)
        print("n di tipo ", elem, ":\t", n, "\tandranno in part1:\t", p8)
        blocco = dataframe[dataframe['dx']==elem]
        blocco1 = blocco.sample(n=p8)
        part1 = part1.append(blocco1)
        part2 = part2.append(blocco.drop(blocco1.index))
    print("size part1: ", len(part1))
    print("size part2: ", len(part2))
    return part1, part2


# recupero dataset originale (senza immagini aumentate) dal dataset completo
def only_orig(dataset):
    reduced = pd.DataFrame()
    for index in range(len(dataset)):
        if len(dataset.iloc[index]["image_id"]) == 12:
            reduced = reduced.append(dataset.iloc[index])
    return reduced

# a seconda della direzione sposta le immagini dentro oppure fuori le cartelle classi (0,...,6) partendo dalla loro madre
def dir_game(dataset, out=0, cartella='..\..\skin-cancer-mnist-ham10000\HAM10000_images_part_1\\'):
    for index in range(len(dataset)):
        img=dataset.iloc[index]["image_id"]
        source = cartella + img +'.jpg'
        dx =str(dataset.iloc[index]["dx"])
        dest = cartella + dx + '\\' + img + '.jpg'
        if out == 0:
            os.replace(source, dest)         # metto le immagini dentro le cartelle classi
        elif out == 1:
            os.replace(dest, source)         # metto le immagini fuori le cartelle classi
        else:
            print("input error")
            break
    return

In [4]:
# variabili globali
#### GIUSEPPE
#image_path = '..\\..\skin-cancer-mnist-ham10000\HAM10000_images_part_1'   #path immagini 600x450
image_path = '..\\..\\Untitled_Folder'      # path immagini 200x150
csv_path = 'HAM10000_metadata.csv'
csv_completo = 'dataframe_completo.csv'     # in 10016 c'è una riga fake
csv_completo_2 = 'dataframe_completo_2.csv'
csv_orig = '/csv/dataframe_originale.csv'   # csv su colab senza immagini aumentate
aug_dir = '..\\..\\aug_dir'                 # path delle immagini aumentate

#### FABIO
# image_path = '..\skin-cancer-mnist-ham10000\HAM10000_images_part_1'

In [5]:
# correlazione standard tra le colonne ############################################NON FUNZIONA
df = pd.read_csv(csv_path, encoding = "ISO-8859-1")
df.corr(method='pearson')

,age
age,1.0


In [6]:
# INIZIALIZZAZIONE

#creazione dataframe in memoria con colonne [image_id, dx]
ds, labels = initialize_dataset()


# label encoding delle labels (dx)
encoder = preproc.LabelEncoder()
encoded = lab_encode(ds, encoder)
encoded = encoded.sort_index()

encoded

,image_id,dx
0,ISIC_0027419,2
1,ISIC_0025030,2
2,ISIC_0026769,2
3,ISIC_0025661,2
4,ISIC_0031633,2
...,...,...
10010,ISIC_0033084,0
10011,ISIC_0033550,0
10012,ISIC_0033536,0
10013,ISIC_0032854,0


In [7]:
# analisi dei valori per data augmentation
aug_size = [0,0,0,0,0,0,0]
stats(encoded, aug_size)

class  0 :	 327
ugmented per image:  19 

class  1 :	 514
ugmented per image:  12 

class  2 :	 1099
ugmented per image:  5 

class  3 :	 115
ugmented per image:  57 

class  4 :	 1113
ugmented per image:  5 

class  5 :	 6705
ugmented per image:  0 

class  6 :	 142
ugmented per image:  46 



In [ ]:
# DATA AUGMENTATION

# Create a data generator
datagen = ImageDataGenerator(
    rotation_range=180,
    width_shift_range=0.1,
    height_shift_range=0.1,
    zoom_range=0.1,
    horizontal_flip=True,
    vertical_flip=True,
    #brightness_range=(0.9,1.1),
    fill_mode='nearest')

##### creazione immagini augmented

# note that we are not augmenting class 'nv', cioè la 5
class_list = ['0','1','2','3','4','6']

img_dir = os.path.join(aug_dir, 'img_dir')    

# Choose a class
for img_class in class_list:    
    # creazione cartelle contenenti le augmented
    dst_class_dir = os.path.join(img_dir, img_class)
    os.mkdir(dst_class_dir)   # cartelle da 0 a 6 dentro aug_dir/img_dir
    
    # list all images in that directory
    src_class_dir = os.path.join('..\\skin-cancer-mnist-ham10000\\HAM10000_images_part_1', img_class)
    img_list = os.listdir(src_class_dir) # qui le immagini sono già nelle proprie cartelle, cioè in skin-cancer-mnist-ham10000/HAM10000_images_part_1/0, ...,6
    print('#img: '+ str(len(img_list)))
    
    # per ogni immagine devo creare le augmented
    for fname in img_list:
        fpath = os.path.join(src_class_dir, fname) # path completo dell'immagine src
        img = Image.open(fpath)  # this is a PIL image
        x = img_to_array(img)  # this is a Numpy array with shape (3, 150, 150)
        x = x.reshape((1,) + x.shape)  # this is a Numpy array with shape (1, 3, 150, 150)
        
        prefix = fname.replace(".jpg", "")
        # the .flow() command below generates batches of randomly transformed images e lo faccio fino a quando non arrivo all'aug_size
        i = 0
        for batch in datagen.flow(x, batch_size=1, save_to_dir=dst_class_dir, save_prefix=prefix , save_format='jpg'):
            i += 1
            if i > aug_size[int(img_class)]:
                break
        del x
        img.close()

In [ ]:
############## aggiornamento nuovo dataset con le immagini aumentate
img_dir = os.path.join(aug_dir, 'img_dir') 
classes = os.listdir(img_dir)                  # elenco delle cartelle (da 0 a 6)

columns = encoded.columns
tupla = pd.DataFrame(columns=columns)          # dataset vuoto, alla fine dei cicli for conterrà tutte le nuove immagini aumentate con dx cartella madre
index = 10015

for elem1 in classes:
    class_dir = os.path.join(img_dir, elem1)
    images = os.listdir(class_dir)
    for elem2 in images:
        elem2 = elem2.replace('.jpg', '')
        tupla = tupla.append({'image_id': elem2, 'dx': int(elem1)}, ignore_index=True)
        
encoded2 = encoded.append(tupla)
encoded2.to_csv(csv_completo, index=False)     # il csv con la riga 10016 fake           

In [ ]:
# rescaling immagini
src_path = "../../skin-cancer-mnist-ham10000/HAM10000_images_part_1"
dst_path = "../../Untitled Folder"
size1=(200, 150)

count = 0

for fname in os.listdir(src_path):
    img_rsz = Image.open(os.path.join(src_path, fname)).resize(size1)
    img_rsz.save(os.path.join(dst_path, fname))
    count = count + 1
    if count % 5000 == 0:
        print("resizate " + str(count) + " immagini")